In [30]:
from urllib.request import urlopen
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC


In [38]:
def subscrape_data(url, driver):

    posted_by, object_type, condition, heating, floor, max_floors, flag_legal, flag_available_immediately, flag_not_last_floor,flag_balcony, flag_elevator, flag_internet, flag_surveliance, flag_air_condition, flag_cable_tv, flag_garage, flag_interphone, number_of_busses, number_of_trams, number_of_trolleys = ([] for _ in range(20))

    driver.get(url)
    time.sleep(1)      # wait page to load, to grab raw values
    page=driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    info = soup.findAll('div',attrs={'class':'col-lg-7 col-md-7 datasheet-features-type'})

    try:
        posted_by.append(info[0].span.text)
    except:
        posted_by.append('')    
    try:
        object_type.append(info[1].span.text)
    except:
        object_type.append('')
    try:
        condition.append(info[2].span.text)
    except:
        condition.append('')
    try: 
        heating.append(info[3].span.text)
    except:
        heating.append('')
    try:
        floor.append(info[4].span.text)
    except:
        floor.append(-1)
    try: 
        max_floors.append(info[5].span.text)
    except:
        max_floors.append(-1)
    try:
        other = soup.findAll('div',attrs={'class':'flags-container'})
        if 'Odmah useljiv' in other[0].text:
            flag_available_immediately.append(1)
        else:
            flag_available_immediately.append(0)
        if 'Uknjižen' in other[0].text:
            flag_legal.append(1)
        else:
            flag_legal.append(0)
        if 'Nije poslednji sprat' in other[0].text:
            flag_not_last_floor.append(1)
        else:
            flag_not_last_floor.append(0)
        if ('Terasa' in other[1].text) or ('Lođa' in other[1].text):
            flag_balcony.append(1)
        else:
            flag_balcony.append(0)
        if 'Lift' in other[1].text:    
            flag_elevator.append(1)
        else:
            flag_elevator.append(0)
        if 'Internet' in other[1].text: 
            flag_internet.append(1)
        else:
            flag_internet.append(0)
        if 'Video nadzor' in other[1].text: 
            flag_surveliance.append(1)
        else:
            flag_surveliance.append(0)
        if 'Klima' in other[1].text:
            flag_air_condition.append(1)
        else:
            flag_air_condition.append(0)
        if 'KATV' in other[1].text:
            flag_cable_tv.append(1)
        else:
            flag_cable_tv.append(0)
        if 'Garaža' in other[1].text:
            flag_garage.append(1)
        else:
            flag_garage.append(0)
        if 'Interfon' in other[1].text:
            flag_interphone.append(1)
        else:
            flag_interphone.append(0)
    except:
        flag_available_immediately.append(-1)
        flag_legal.append(-1)
        flag_not_last_floor.append(-1)
        flag_balcony.append(-1)    
        flag_elevator.append(-1)
        flag_internet.append(-1)
        flag_surveliance.append(-1)
        flag_air_condition.append(-1)
        flag_cable_tv.append(-1)
        flag_garage.append(-1)
        flag_interphone.append(-1)
    try:    
        number_of_busses.append(len(soup.findAll('div',attrs={'class':'city-lines'})[0].findAll('li',attrs={'class':'busses'})))
        number_of_trams.append(len(soup.findAll('div',attrs={'class':'city-lines'})[0].findAll('li',attrs={'class':'trams'})))
        number_of_trolleys.append(len(soup.findAll('div',attrs={'class':'city-lines'})[0].findAll('li',attrs={'class':'trolleybus'})))  
    except:
        number_of_busses.append(-1)
        number_of_trams.append(-1)
        number_of_trolleys.append(-1)
    df = pd.DataFrame(list(zip(posted_by, object_type, condition, heating, floor, max_floors, flag_legal, flag_available_immediately, flag_not_last_floor,flag_balcony, flag_elevator, flag_internet, flag_surveliance, flag_air_condition, flag_cable_tv, flag_garage, flag_interphone, number_of_busses, number_of_trams, number_of_trolleys)),columns =['Posted_By', 'Object_Type','Condition','Heating_Type','Floor','Max_Floors','Flag_Legal','Flag_Available_Immediately','Flag_Not_Last_Floor','Flag_Balcony','Flag_Elevator','Flag_Internet','Flag_Surveliance','Flag_Air_Condition','Flag_Cable_TV','Flag_Garage','Flag_Interphone','Num_of_Busses','Num_of_Trams','Num_of_Trolleys'])
    
    return df

In [39]:
def scrape_data(base_url, search_addon, page, ad_type,driver):
    
    additional_facts = pd.DataFrame(columns =['Posted_By', 'Object_Type','Condition','Heating_Type','Floor','Max_Floors','Flag_Legal','Flag_Available_Immediately','Flag_Last_Floor','Flag_Balcony','Flag_Elevator','Flag_Internet','Flag_Surveliance','Flag_Air_Condition','Flag_Cable_TV','Flag_Garage','Flag_Interphone','Num_of_Busses','Num_of_Trams','Num_of_Trolleys'])

    url = base_url+search_addon+str(page)
    raw_page = urlopen(url).read()
    urlopen(url).close()
    bs_page = BeautifulSoup(raw_page, 'html.parser')
    
    divs = bs_page.findAll('div',{'class','product-item product-list-item {} real-estates my-ad-placeholder'.format(ad_type)})
    # initialize empty lists
    publish_date, prices, currency, city, city_area, city_subarea, city_street, property_type, property_size, property_norooms, desc, link_to_property   = ([] for _ in range(12))

    if divs: # if list is not empty - to speed things up for empty segments
        for div in divs:

            publish_date.append(div.findAll('div',{'class','pi-img-wrapper-under'})[0].span.text)       # extract date of ad posting

            price = div.findAll('div',{'data-value','central-feature'})[0].span.text                    # price and currency info
            price = price.split('\xa0')

            prices.append(int(price[0].translate(str.maketrans({'.': '', ',': ''}))))
            currency.append(price[1])

            city_info = div.findAll('div',{'class','col-md-6 col-sm-5 col-xs-6 col-lg-6 sm-margin'})[0] # extract city, area and street

            city.append(city_info.find('ul',attrs={'class':'subtitle-places'}).text.split('\xa0')[0])
            city_area.append(city_info.find('ul',attrs={'class':'subtitle-places'}).text.split('\xa0')[1])
            city_subarea.append(city_info.find('ul',attrs={'class':'subtitle-places'}).text.split('\xa0')[2])
            city_street.append(city_info.find('ul',attrs={'class':'subtitle-places'}).text.split('\xa0')[3])

            features = div.find('ul',attrs={'class':'ad-features'})                                     # extract price, size and type of posting

            property_type.append(features.findAll('li',{'class':'col-p-1-3'})[0].div.text.split('\xa0')[0])
            property_size.append(features.findAll('li',{'class':'col-p-1-3'})[1].div.text.split('\xa0')[0])
            property_norooms.append(features.findAll('li',{'class':'col-p-1-3'})[2].div.text.split('\xa0')[0])

            try: 
                desc.append(div.findAll('p',{'class':'text-description-list ad-description short-desc'})[0].text)
            except:
                desc.append('')
            link_to_property.append(div.find('h3').a['href'])
    #        print(link_to_property[len(link_to_property)-1])
            tmp_additional_facts = subscrape_data(str(base_url+link_to_property[len(link_to_property)-1]),driver)

            additional_facts = additional_facts.append(tmp_additional_facts, ignore_index=True)

        df = pd.DataFrame(list(zip(publish_date,prices,currency,city,city_area,city_subarea,city_street,property_type,property_size,property_norooms,desc, link_to_property)), 
               columns =['Publish_Date', 'Price','Currency','City','City_Area','City_Subarea','Street','Property_Type','Property_Size_sqmtr','No_of_Rooms','Desc','Link_to_Property'])
        df['Ad_Type'] = ad_type
    else:
        df = pd.DataFrame(list(zip(publish_date,prices,currency,city,city_area,city_subarea,city_street,property_type,property_size,property_norooms,desc, link_to_property)), 
               columns =['Publish_Date', 'Price','Currency','City','City_Area','City_Subarea','Street','Property_Type','Property_Size_sqmtr','No_of_Rooms','Desc','Link_to_Property'])
        df['Ad_Type'] = None

    return pd.concat([df, additional_facts], axis=1,sort=True)

In [47]:
num_pages = 1473

for page in range(1,num_pages):
    driver = Chrome('chromedriver1.exe')
    print('page: ' + str(page))
    tmp_df = scrape_data('https://www.halooglasi.com', '/nekretnine/prodaja-stanova?page=', page, 'Premium',driver)
    print(len(tmp_df))
    df = pd.concat([df, tmp_df], axis=0, ignore_index=True,sort=True)
    tmp_df = scrape_data('https://www.halooglasi.com', '/nekretnine/prodaja-stanova?page=', page, 'Top',driver)
    print(len(tmp_df))
    df = pd.concat([df, tmp_df], axis=0, ignore_index=True,sort=True)
    tmp_df = scrape_data('https://www.halooglasi.com', '/nekretnine/prodaja-stanova?page=', page, 'Standard',driver)
    print(len(tmp_df))
    df = pd.concat([df, tmp_df], axis=0, ignore_index=True,sort=True)
    driver.close()

In [41]:
df.drop_duplicates().to_csv('halooglasi_data_20191102_v2.csv',index=False)

In [48]:
df = pd.read_csv('halooglasi_data_20191102_v2.csv')
df = df.iloc[1:]

In [45]:
df.head()

,Ad_Type,City,City_Area,City_Subarea,Condition,Currency,Desc,Flag_Air_Condition,Flag_Available_Immediately,Flag_Balcony,...,Num_of_Busses,Num_of_Trams,Num_of_Trolleys,Object_Type,Posted_By,Price,Property_Size_sqmtr,Property_Type,Publish_Date,Street
1,Premium,Beograd,Opština Zvezdara,Mali Mokri Lug,Izvorno stanje,€,Stambeni objekat se nalazi u Partizanskoj ulic...,1,1,1,...,9,0,0,Novogradnja,Investitor,45000.0,"36,39",Stan,02.11.2019,Partizanska
2,Premium,Beograd,Opština Zvezdara,Mali Mokri Lug,Izvorno stanje,€,Stambeni objekat se nalazi u Partizanskoj ulic...,1,1,1,...,9,0,0,Novogradnja,Investitor,50150.0,"40,15",Stan,02.11.2019,Partizanska
3,Premium,Beograd,Opština Grocka,Kaluđerica,Izvorno stanje,€,"Stan se nalazi u Kaluđerici, u blizini Doma zd...",1,1,1,...,8,0,0,Novogradnja,Vlasnik,32000.0,32,Stan,02.11.2019,NaN
4,Premium,Beograd,Opština Grocka,Kaluđerica,Izvorno stanje,€,"Stan se nalazi u naselju Kaluđerica, kod pijac...",0,1,1,...,1,0,0,Novogradnja,Investitor,33500.0,"37,24",Stan,02.11.2019,Kopaonička
5,Premium,Beograd,Opština Zvezdara,Mirijevo,Izvorno stanje,€,"Zgrada se nalazi u naselju Mirijevo, ulica Mih...",-1,0,-1,...,5,0,0,Novogradnja,Investitor,49540.0,"45,04",Stan,02.11.2019,Mihaila Bulgakova
